**Only 360 after removing BC**

In [1]:
import pandas as pd
import requests
from IPython.display import Image
import time
from ftplib import FTP

from PIL import Image
from PIL import ImageFile
import rawpy
import imageio
ImageFile.LOAD_TRUNCATED_IMAGES = True


import os, shutil

import os
import sys

In [60]:
imagecount = 72

In [61]:
typdict = {'NDC':'str'}
df = pd.read_csv('../data/directory_of_images.txt',sep='|', dtype=typdict, names=['NDC','IDK','FILE','TYPE','DRUG'])
df = df.dropna()
df.DRUG = df.DRUG.str.upper()
df[['ORIG_FOLDER','IMAGES','FILENAME']] = df.FILE.str.split('/', expand=True)
df['FILETYPE'] = df.FILENAME.str[-4:]
df = df[df.FILETYPE != '.WMV']

In [62]:
ndcs = df.NDC.value_counts()
n = ndcs[ndcs == imagecount].index
# #n100 = ndcs[ndcs <= 100].index
# ndcs50100 = list(set(n50) & set(n100))
dfn = df[df.NDC.isin(n)]
dfn

,NDC,IDK,FILE,TYPE,DRUG,ORIG_FOLDER,IMAGES,FILENAME,FILETYPE
15680,00093106301,1,PillProjectDisc108/images/Z1YM4S_7V554NEH9Z3FO...,C3PI_Reference,SOTALOL HYDROCHLORIDE,PillProjectDisc108,images,Z1YM4S_7V554NEH9Z3FOEGOAM0USK.CR2,.CR2
15681,00093106301,1,PillProjectDisc10/images/741HJNKDE38QP851XAI3V...,C3PI_Reference,SOTALOL HYDROCHLORIDE TABLETS USP,PillProjectDisc10,images,741HJNKDE38QP851XAI3VA!!COLIZH.CR2,.CR2
15682,00093106301,1,PillProjectDisc30/images/BE!_XZRE9OTI4SB9!UY3I...,C3PI_Reference,SOTALOL HYDROCHLORIDE,PillProjectDisc30,images,BE!_XZRE9OTI4SB9!UY3I07IDD7Z-YB.CR2,.CR2
15683,00093106301,1,PillProjectDisc43/images/BQ250UT34T3XHGF853T2M...,C3PI_Reference,SOTALOL HYDROCHLORIDE TABLETS USP,PillProjectDisc43,images,BQ250UT34T3XHGF853T2M_6MEW56NIK.CR2,.CR2
15684,00093106301,1,PillProjectDisc100/images/RIQ!QL_32B!M_QM22IZ!...,C3PI_Test,SOTALOL HYDROCHLORIDE,PillProjectDisc100,images,RIQ!QL_32B!M_QM22IZ!RCN4OU_QBE.JPG,.JPG
...,...,...,...,...,...,...,...,...,...
96463,51285009287,1,PillProjectDisc72/images/COPAPY009V!7B90H3JEOB...,MC_COOKED_CALIBRATED_V1.2,LOSEASONIQUE,PillProjectDisc72,images,COPAPY009V!7B90H3JEOBZTIC--8YE!.PNG,.PNG
96464,51285009287,2,PillProjectDisc83/images/CYJ!LDYGEV__282!-_ZUZ...,MC_COOKED_CALIBRATED_V1.2,LOSEASONIQUE,PillProjectDisc83,images,CYJ!LDYGEV__282!-_ZUZCMMWCK!W4R.PNG,.PNG
96465,51285009287,2,PillProjectDisc98/images/P26AQOOMGA-0X85GNXWG3...,MC_COOKED_CALIBRATED_V1.2,LOSEASONIQUE,PillProjectDisc98,images,P26AQOOMGA-0X85GNXWG38AE_HC1MP.PNG,.PNG
96466,51285009287,1,PillProjectDisc39/images/BMGFDA72PPUN015YB95KA...,MC_SPL_SPLIMAGE_V3.0,LOSEASONIQUE,PillProjectDisc39,images,BMGFDA72PPUN015YB95KANIRM2JIWWB.JPG,.JPG


In [63]:
dfn.DRUG.value_counts()

JUNEL FE 1/20                             72
SEASONIQUE                                72
LOSEASONIQUE                              72
PREFEST                                   72
DICLOXACILLIN SOD 250MG CAPS              42
SOTALOL HYDROCHLORIDE                     38
METOCLOPRAMIDE                            38
BACLOFEN TABLETS USP                      36
CLINDAMYCIN HYDROCHLORIDE                 36
CLINDAMYCIN HYDROCHLORIDE CAPSULES USP    36
BACLOFEN                                  36
METOCLOPRAMIDE TABLETS, USP               34
SOTALOL HYDROCHLORIDE TABLETS USP         34
DICLOXACILLIN SODIUM CAPSULES USP         30
Name: DRUG, dtype: int64

In [64]:
drugs = dfn.NDC.value_counts()
drugslist = drugs.index.tolist()
len(dfn.NDC.value_counts()),drugs

(9,
 00093220401    72
 00093312301    72
 51285008787    72
 00093106301    72
 51285006390    72
 00555902658    72
 00172409660    72
 51285009287    72
 00093525668    72
 Name: NDC, dtype: int64)

In [65]:
df_a = dfn.groupby(['DRUG','NDC']).count()
df_a.reset_index(inplace=True)
drug_ndc = df_a[['DRUG','NDC']]
drug_ndc

,DRUG,NDC
0,BACLOFEN,00172409660
1,BACLOFEN TABLETS USP,00172409660
2,CLINDAMYCIN HYDROCHLORIDE,00093525668
3,CLINDAMYCIN HYDROCHLORIDE CAPSULES USP,00093525668
4,DICLOXACILLIN SOD 250MG CAPS,00093312301
5,DICLOXACILLIN SODIUM CAPSULES USP,00093312301
6,JUNEL FE 1/20,00555902658
7,LOSEASONIQUE,51285009287
8,METOCLOPRAMIDE,00093220401
9,"METOCLOPRAMIDE TABLETS, USP",00093220401


In [66]:
droplist = ['00555902658','51285009287','51285006390','51285008787']

df = dfn[~dfn.NDC.isin(droplist)]

In [67]:
df.DRUG[df.NDC == '00172409660'] = 'BACLOFEN'
df.DRUG[df.NDC == '00093525668'] = 'CLINDAMYCIN'
df.DRUG[df.NDC == '00093312301'] = 'DICLOXACILLIN SOD 250MG'
df.DRUG[df.NDC == '00093220401'] = 'METOCLOPRAMIDE'
df.DRUG[df.NDC == '00093106301'] = 'SOTALOL HYDROCHLORIDE'

/Users/alphonsowoodbury/.conda/envs/aw0520/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/alphonsowoodbury/.conda/envs/aw0520/lib/python3.7/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/alphonsowoodbury/.conda/envs/aw0520/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [68]:
df

,NDC,IDK,FILE,TYPE,DRUG,ORIG_FOLDER,IMAGES,FILENAME,FILETYPE
15680,00093106301,1,PillProjectDisc108/images/Z1YM4S_7V554NEH9Z3FO...,C3PI_Reference,SOTALOL HYDROCHLORIDE,PillProjectDisc108,images,Z1YM4S_7V554NEH9Z3FOEGOAM0USK.CR2,.CR2
15681,00093106301,1,PillProjectDisc10/images/741HJNKDE38QP851XAI3V...,C3PI_Reference,SOTALOL HYDROCHLORIDE,PillProjectDisc10,images,741HJNKDE38QP851XAI3VA!!COLIZH.CR2,.CR2
15682,00093106301,1,PillProjectDisc30/images/BE!_XZRE9OTI4SB9!UY3I...,C3PI_Reference,SOTALOL HYDROCHLORIDE,PillProjectDisc30,images,BE!_XZRE9OTI4SB9!UY3I07IDD7Z-YB.CR2,.CR2
15683,00093106301,1,PillProjectDisc43/images/BQ250UT34T3XHGF853T2M...,C3PI_Reference,SOTALOL HYDROCHLORIDE,PillProjectDisc43,images,BQ250UT34T3XHGF853T2M_6MEW56NIK.CR2,.CR2
15684,00093106301,1,PillProjectDisc100/images/RIQ!QL_32B!M_QM22IZ!...,C3PI_Test,SOTALOL HYDROCHLORIDE,PillProjectDisc100,images,RIQ!QL_32B!M_QM22IZ!RCN4OU_QBE.JPG,.JPG
...,...,...,...,...,...,...,...,...,...
35555,00172409660,1,PillProjectDisc65/images/CHJQYY117S90R9O-8B7KL...,MC_COOKED_CALIBRATED_V1.2,BACLOFEN,PillProjectDisc65,images,CHJQYY117S90R9O-8B7KLCWKYX9RO5T.PNG,.PNG
35556,00172409660,1,PillProjectDisc81/images/CWSHIOSDVUT5ZH95HISRC...,MC_COOKED_CALIBRATED_V1.2,BACLOFEN,PillProjectDisc81,images,CWSHIOSDVUT5ZH95HISRCU_GGBEO7D2.PNG,.PNG
35557,00172409660,1,PillProjectDisc87/images/F90B1UFT0CYEQAUY1WYRF...,MC_COOKED_CALIBRATED_V1.2,BACLOFEN,PillProjectDisc87,images,F90B1UFT0CYEQAUY1WYRF-8PSVC2L7.PNG,.PNG
35558,00172409660,1,PillProjectDisc7/images/43L-7N45KZG_YF2_3BJK!-...,MC_SPL_SPLIMAGE_V3.0,BACLOFEN,PillProjectDisc7,images,43L-7N45KZG_YF2_3BJK!-N1K9LAD9.JPG,.JPG


In [69]:
df.DRUG.value_counts()

SOTALOL HYDROCHLORIDE      72
DICLOXACILLIN SOD 250MG    72
CLINDAMYCIN                72
METOCLOPRAMIDE             72
BACLOFEN                   72
Name: DRUG, dtype: int64

In [70]:
df_a = df.groupby(['DRUG','NDC']).count()
df_a.reset_index(inplace=True)
drug_ndc = df_a[['DRUG','NDC']]
drug_ndc

,DRUG,NDC
0,BACLOFEN,00172409660
1,CLINDAMYCIN,00093525668
2,DICLOXACILLIN SOD 250MG,00093312301
3,METOCLOPRAMIDE,00093220401
4,SOTALOL HYDROCHLORIDE,00093106301


In [31]:
drugsftp = df.FILE.tolist()
drugsftp.sort()
drugsftp[:10]

['PillProjectDisc1/images/!56HENXQPITL7C!4A52GSM0YNF5KKA.PNG',
 'PillProjectDisc1/images/!7WF161YS_R5ET9Y!26E6UHXF9HXDU.JPG',
 'PillProjectDisc1/images/!G_6-MR6ALJIVSJV_FG81XV_T76S5C.JPG',
 'PillProjectDisc1/images/!KIVP9GRT_9CCE7MKA2-!EUSBJK44B.PNG',
 'PillProjectDisc1/images/!WT4Y657!VOLO3YRBJ5HS9MSRVMZKE.JPG',
 'PillProjectDisc10/images/6_6!XJ73P9O12E6LVGBAIIGMNO252A.JPG',
 'PillProjectDisc10/images/6_6!XJ73P9O12E6LVGBAIIGMNO252A.JPG',
 'PillProjectDisc10/images/71F0PADPK6212-1HT3-IBI-Y!KK!1B.JPG',
 'PillProjectDisc10/images/71QVB659EJF7J2UOJAR9107P-CC6GM.JPG',
 'PillProjectDisc10/images/71QVB659EJF7J2UOJAR9107P-CC6GM.JPG']

In [32]:
ftpdir = {}
for i in range(0,len(drugsftp)):
    ftpcall = drugsftp[i].split('/')
    ftppath = ftpcall[0]+'/'+ftpcall[1]+ '/'
    ftpfile = ftpcall[-1]
    ftpdir.setdefault(ftppath,[])
    ftpdir[ftppath].append(ftpfile)
len(ftpdir)

102

In [ ]:
%cd /content/drive/My Drive/Project: Pill ID/Data
for key in ftpdir.keys():
    #%cd /content/drive/My Drive/Project: Pill ID/Data
    ftp = FTP('lhcftp.nlm.nih.gov')
    ftp.login()
    ftp.cwd('Open-Access-Datasets/Pills//')
    ftp.cwd(key)
    dloaded = os.listdir('/content/drive/My Drive/Project: Pill ID/Data')
    print(key)
    for value in ftpdir[key]:
      if value not in dloaded:
        with open(value, 'wb') as fp:
            ftp.retrbinary('RETR '+ value, fp.write)
    ftp.cwd('..')
    ftp.cwd('..')